In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [ ]:
# 관측소 코드 담기

where = 47122   # 오산

In [2]:
where = 47102   # 백령도

In [ ]:
final = pd.DataFrame()

# 30일, 31일 월 분리
# 2월은 따로 생각해야 함
months_30 = [4, 6, 9, 11]
months_31 = [1, 3, 5, 7, 8, 10, 12]

for year in range(2014, 2022):   # 여기 바꾸기
    for month in range(1, 13): 
        if month in months_30: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3012&STNM={}'.format(year, month, where), verify=False)
        elif month in months_31: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3112&STNM={}'.format(year, month, where), verify=False)
        else:   # 2월인 경우
            if year % 4 == 0:   # 윤년인 경우 따로 분리 - 29일까지 있음
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2912&STNM={}'.format(year, month, where), verify=False)
            else:   # 28일까지 있는 경우
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2812&STNM={}'.format(year, month, where), verify=False)
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.select('body > h2')  # title은 하나씩 있음
        data = soup.select('body > pre')  # pre는 하나 건너 하나씩 concat 해야함

        idx_data = 0
        for idx in range(len(title)): 
            data_df = pd.DataFrame(data[idx_data])
            data_df[0] = data_df[0].str.replace("-----------------------------------------------------------------------------", "")
            data_df_2 = data_df[0].str.split('\n', expand=True)

            data_df_2.drop([0, 1, 2, 3, 4], axis = 1, inplace = True)

            data_df_2 = data_df_2.transpose()
            title_tmp = str(title[idx])
            data_df_2['date'] = title_tmp[4:56]   # 백령도 기준 4:56, 오산 기준 4:54
            
            final = pd.concat([final, data_df_2], axis = 0)

            idx_data += 2

In [ ]:
# 백령도 14 ~ 21 돌리는 중

In [9]:
title_tmp[4:56]

'47102  Baengnyeongdo Observations at 12Z 31 Dec 2021'

In [10]:
final

,0,date
5,993.0 158 4.6 -2.4 60 3.24 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
6,978.0 283 4.0 -4.0 56 2.92 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
7,925.0 738 -0.3 -4.5 73 2.97 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
8,901.0 948 -2.5 -5.1 82 2.92 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
9,872.0 1207 -4.1 -17.1 36 1.15 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
...,...,...
56,61.0 19183 -58.7 -80.7 4 0.01 ...,47102 Baengnyeongdo Observations at 12Z 31 De...
57,58.0 19501 -57.7 -79.7 4 0.01 ...,47102 Baengnyeongdo Observations at 12Z 31 De...
58,54.3 19916 -56.3 -78.3 5 0.02 ...,47102 Baengnyeongdo Observations at 12Z 31 De...
59,54.0 ...,47102 Baengnyeongdo Observations at 12Z 31 De...


In [11]:
final2 = final.reset_index(drop = True)
final2

,0,date
0,993.0 158 4.6 -2.4 60 3.24 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
1,978.0 283 4.0 -4.0 56 2.92 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
2,925.0 738 -0.3 -4.5 73 2.97 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
3,901.0 948 -2.5 -5.1 82 2.92 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
4,872.0 1207 -4.1 -17.1 36 1.15 ...,47102 Baengnyeongdo Observations at 00Z 01 Ja...
...,...,...
514376,61.0 19183 -58.7 -80.7 4 0.01 ...,47102 Baengnyeongdo Observations at 12Z 31 De...
514377,58.0 19501 -57.7 -79.7 4 0.01 ...,47102 Baengnyeongdo Observations at 12Z 31 De...
514378,54.3 19916 -56.3 -78.3 5 0.02 ...,47102 Baengnyeongdo Observations at 12Z 31 De...
514379,54.0 ...,47102 Baengnyeongdo Observations at 12Z 31 De...


In [12]:
final2.to_csv('D:\\SNUlab\\Baengnyeongdo_crawling_1401_2112.csv', encoding = 'euc-kr', index = False)